In [ ]:
#import torch and device

import torch
from torch import nn
import torchvision

if( torch.cuda.is_available()):
    device="cuda"
else:
    device="cpu"
device

In [ ]:
!nvidia-smi

In [ ]:
print("hello")

In [ ]:
import zipfile
from pathlib import Path

data_path= Path("../archive")
image_path = data_path / ""

In [ ]:
from torchvision import datasets, transforms

path="C:\\Desktop files\\amaan files\\BMS stuff\\4th SEM\\python_programming\\python_project\\New Plant Diseases Dataset(Augmented)"

transforms = transforms.Compose([
                                      transforms.CenterCrop(256),
                                      transforms.ToTensor()])

train_data=datasets.ImageFolder(path+f'/train',transform=transforms)
test_data=datasets.ImageFolder(path+f'/valid',transform=transforms)

In [ ]:
BATCH_SIZE=64
train_dataloader=torch.utils.data.DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_dataloader=torch.utils.data.DataLoader(test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
images,labels=next(iter(train_dataloader))

#image=mpimg.imread(train_data[0][0])
plt.imshow(torch.permute(train_data[10][0],(1,2,0)))
plt.show()

In [ ]:
class_names=train_data.classes

In [ ]:
torch.manual_seed(42)

def eval_model(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,accuracy_fn):
    loss,acc=0,0
    model.eval()
    with torch.inference_mode():
        for X,y in data_loader:
            y_pred= model(X.to(device))

            loss+= loss_fn(y_pred.to(device),y.to(device))
            acc+= accuracy_fn(y_true=y.to(device),y_pred=y_pred.argmax(dim=1).to(device))
        
        # scale loss and acc to find average loss/acc per batch

        loss/=len(data_loader)
        acc/=len(data_loader)
    return {"model_name":model.__class__.__name__,# only works when model was created with class},
            "model_loss":loss.item(),
            "model_acc":acc}

In [ ]:
len(train_dataloader)

In [ ]:
class CNN(nn.Module):

    """tiny VGG"""

    def __init__(self, input_shape:int, hidden_units: int, output_shape: int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1), 
                            nn.SiLU(),
                            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1),
                            nn.SiLU(),
                            nn.MaxPool2d(2))
        self.conv_block_2=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1), 
                            nn.SiLU(),
                            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1),
                            nn.SiLU(),
                            nn.MaxPool2d(2)
            
        )
        self.conv_block_3=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1), 
                            nn.SiLU(),
                            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1),
                            nn.SiLU(),
                            nn.MaxPool2d(2)
            
        )
        self.conv_block_4=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1), 
                            nn.SiLU(),
                            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1),
                            nn.SiLU(),
                            nn.MaxPool2d(2)
            
        )
        self.conv_block_5=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1), 
                            nn.SiLU(),
                            nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units,
                            kernel_size=3,stride=1,padding=1),
                            nn.SiLU(),
            
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*16*16,out_features=hidden_units),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.Linear(in_features=hidden_units,out_features=hidden_units),
            nn.Linear(in_features=hidden_units,out_features=output_shape),        
        )
    def forward(self,x):
        x=self.conv_block_1(x)
        #print(x.shape)
        x=self.conv_block_2(x)
        #print(x.shape)
        x=self.conv_block_3(x)
        x=self.conv_block_4(x)
        x=self.conv_block_5(x)
        x=self.classifier(x)
        return x

In [ ]:
#del model_2

In [ ]:
torch.manual_seed(42)

model_2= CNN(input_shape=train_data[0][0].shape[0],hidden_units=32,output_shape=len(class_names))
model_2.to('cuda')

In [ ]:
torch.manual_seed(42)

images = torch.randn(size=(BATCH_SIZE,3,256,256))
test_image = images[0]
print(f"Image Batch Shape: {images.shape}")
print(f"Single image shape: {test_image.shape}")
print(f"Tests image: \n {test_image}")
""" print(f"data image: \n {image}") """

In [ ]:
l1=nn.Sequential(nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,padding=1,stride=1),
                 nn.ReLU(),
nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,padding=1,stride=1),
nn.ReLU())
l2=nn.MaxPool2d(2)
x=l1(images)
x=l2(x)
#x=l1(x)
#x=l2(x)
x.shape

In [ ]:
l1=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,padding=1,stride=1)
,nn.ReLU(),
nn.Conv2d(in_channels=10,out_channels=10,kernel_size=3,padding=1,stride=1)
,nn.ReLU())
l2=nn.MaxPool2d(2)
print(x.shape)
x=l1(x)
x=l2(x)
print(x.shape)
x=l1(x)
x=l2(x)
print(x.shape)
l4=nn.Flatten()
l4(x).shape

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    model.train()
    gb=1024*3
    for batch, (X, y) in enumerate(data_loader):
        # Send data to GPU
        
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)
        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        acc=accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels
        train_acc += acc
        # 3. Optimizer zero grad


        # 4. Loss backward
        loss.backward()
        loss.detach()
        # 5. Optimizer step
        optimizer.step()
        optimizer.zero_grad()
        print(f"train_loss : {train_loss/(batch+1)}")
        if(batch%100==0):
            print(f"Batch: {batch} ")
            print(f"Mem: {torch.cuda.memory_allocated()/gb}")
            
        del loss,y_pred

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

@torch.inference_mode()
def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
     
    for X, y in data_loader:
        # Send data to GPU
        X, y = X.to(device), y.to(device)
        
        # 1. Forward pass
        test_pred = model(X)
        
        # 2. Calculate loss and accuracy
        test_loss += loss_fn(test_pred, y)
        test_acc += accuracy_fn(y_true=y,
            y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
        )
        
    # Adjust metrics and print out
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
def accuracy_fn(y_true,y_pred):
    correct=torch.eq(y_true,y_pred).sum().item()
    acc= (correct/len(y_pred))*100
    return acc

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_2.parameters(),lr=0.01)

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from timeit import default_timer as Timer
from tqdm.auto import tqdm

#train and test model

epochs=10

for epoch in tqdm(range(epochs)):
    print(f"Epoch : {epoch}\n-------")

    train_step(
        model=model_2,
        data_loader=train_dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn,
        device=device
    )
    test_step(
        model=model_2,
        data_loader=test_dataloader,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn,
        device=device
    )

In [ ]:
total_params = sum(p.numel() for p in model_2.parameters())
total_params

In [ ]:

model_2_results = eval_model(
    model=model_2,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn
)
model_2_results

In [ ]:
torch.save(model_2.state_dict(),"disease_model.pth")